In [1]:
%%capture
!pip install neptune-client psutil
!git clone https://github.com/Cho-D-YoungRae/URP_PD.git
%cd URP_PD

In [2]:
import dataset
import object_detection
from utils import *
import eval

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.utils import make_grid
import torchvision.transforms.functional as TF
import torch.nn.functional as F

import os
import json
import numpy as np
import argparse
from tqdm.auto import tqdm
import time
from datetime import datetime

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using <{device}> device")

loading annotations into memory...
Done (t=0.06s)
creating index...
index created!
Using <cuda> device


In [3]:
# ====== constants ======#
label_map = {'background': 0, 'person': 1}
rev_label_map = {v: k for k, v in label_map.items()} 

## setting

In [4]:
from torch.backends import cudnn
import random

cudnn.benchmark = True

# ====== Random Seed Initialization ====== #
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
random.seed(seed)

# torch.cuda.manual_seed(seed)
# torch.cuda.manual_seed_all(seed)
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = False

parser = argparse.ArgumentParser()
args = parser.parse_args("")
args.baselineID = 36

# ====== Dataset ====== #
args.ch_option = {'num_ch': 1,
                  'img_type': 'lwir',
                  'one_ch_option': 'mean'}
args.val_split = 0.1

# ====== Model ====== #
args.base_model = 'VGG16bnBase'
args.n_classes = len(label_map)
args.is_sds = True
args.usages_seg_feats = [True, True, True, False, False, False]
args.get_separate_loss = True


# ====== Optimizer & Training ====== #
args.optim = 'Adam'
args.lr = 5e-4
args.twice_b_lr = True
args.weight_decay = 5e-4

args.epochs = 150
args.train_batch_size = 32
args.test_batch_size = 64

args.decay_lr_at = [int(args.epochs/6)*4,
                    int(args.epochs/6)*5]
args.decay_lr_to = 0.1

## neptune init

In [5]:
import neptune.new as neptune

api_token = 
run = neptune.init(
    project='jodyr/urp',
    source_files=['*.py'],
    api_token=api_token,
    # run='PD-25',
    )

run["parameters"] = vars(args)


https://app.neptune.ai/jodyr/urp/e/PD-93
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


## train

In [6]:
from neptune.new.types import File

def train(train_loader, model, criterion, optimizer, run=None):
    """
    One epoch's training.

    :param train_loader: DataLoader for training data
    :param model: model
    :param criterion: MultiBox loss
    :param optimizer: optimizer
    :param epoch: epoch number
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    model.train()  # training mode enables dropout

    batch_time = AverageMeter()  # forward prop. + back prop. time
    data_time = AverageMeter()  # data loading time
    losses = AverageMeter()  # loss

    if criterion.get_separate_loss:
        sep_losses = {'conf_loss': AverageMeter(),
                      'loc_loss': AverageMeter(),
                      'total_seg_loss': AverageMeter()}

    print_freq = len(train_loader) // 4

    start = time.time()

    # Batches
    for i, (images, bboxes, category_ids, is_crowds, seg_labels) in enumerate(train_loader):
        data_time.update(time.time() - start)

        images = images.to(device)  # (batch_size (N), 3, 300, 300)
        bboxes = [b.to(device) for b in bboxes]
        category_ids = [c.to(device) for c in category_ids]
        seg_labels = seg_labels.to(device)

        # Forward prop.
        predicted_locs, predicted_scores, pred_segs = model(images)  # (N, 8732, 4), (N, 8732, n_classes)

        if criterion.get_separate_loss:
            loss, separate_loss = criterion(
                predicted_locs, predicted_scores, bboxes,
                category_ids,pred_segs, seg_labels)  # scalar
            for loss_name, sep_loss in separate_loss.items():
                sep_losses[loss_name].update(sep_loss, images.size(0))

        else:
            loss = criterion(predicted_locs, predicted_scores, 
                             bboxes, category_ids, pred_segs, seg_labels)  # scalar

        # Backward prop.
        optimizer.zero_grad()
        loss.backward()

        # Update model
        optimizer.step()

        losses.update(loss.item(), images.size(0))
        batch_time.update(time.time() - start)

        start = time.time()
        
        if i == 0 and run:
            ft_names = ['conv4_3', 'conv7', 'conv8_2', 'conv9_2', 'conv10_2', 'conv11_2']
            for ft_name, pred_seg in zip(ft_names, [pred_seg.detach() 
                                                    for pred_seg in pred_segs]):
                pred_seg = F.softmax(pred_seg, dim=1)
                pred_seg_grid = make_grid(pred_seg)
                pred_seg_grid = TF.to_pil_image(pred_seg_grid[1])
                run[f'train/pred_seg_{ft_name}'].log(File.as_image(pred_seg_grid))

            img_grid = make_grid(images)
            img_grid = TF.to_pil_image(img_grid)
            run['train/input'].log(File.as_image(img_grid))
            seg_gt_grid = make_grid(seg_labels)
            seg_gt_grid = TF.to_pil_image(seg_gt_grid)
            run['train/seg_gt'].log(File.as_image(seg_gt_grid))

        # Print status
        if i % print_freq == 0:
            print(f'[{i}/{len(train_loader)}]\t'
                  f'Batch Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  f'Data Time {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  f'Loss {losses.val:.4f} ({losses.avg:.4f})\t')
    del predicted_locs, predicted_scores, images, bboxes, category_ids

    if run:
        run['train/loss'].log(losses.avg)
        if criterion.get_separate_loss:
            for loss_name, sep_loss in sep_losses.items():
                run[f'train/{loss_name}'].log(sep_loss.avg)

## validation

In [7]:
# def validation(val_loader, model, criterion):
#     model.eval()

#     num_batches = len(val_loader)
#     losses = AverageMeter()
#     with torch.no_grad():
#         for i, (images, bboxes, category_ids, _) in enumerate(val_loader):
#             images = images.to(device)
#             bboxes = [b.to(device) for b in bboxes]
#             category_ids = [l.to(device) for l in category_ids]

#             predicted_locs, predicted_scores = model(images)
#             loss = criterion(predicted_locs, predicted_scores, bboxes, category_ids).item()

#             losses.update(loss, images.size(0))

#     val_loss = losses.avg
#     return val_loss

## checkpoint

In [8]:
checkpoint = os.path.join('/content/drive/MyDrive/2021.summer_URP/PD/checkpoint',
                          str(args.baselineID)+'.pth.tar')
checkpoint = checkpoint if os.path.isfile(checkpoint) else None
print(f"checkpoint: {checkpoint}")

checkpoint: None


In [9]:
if checkpoint is None:
    start_epoch = 1
    lr = args.lr
    model = object_detection.SDSSSD300(n_classes=args.n_classes,
                                       base=args.base_model,
                                       ch_option=args.ch_option,
                                       usages_seg_feats=args.usages_seg_feats)
    if args.twice_b_lr:
        biases = list()
        not_biases = list()
        for param_name, param in model.named_parameters():
            if param.requires_grad:
                if param_name.endswith('.bias'):
                    biases.append(param)
                else:
                    not_biases.append(param)
        optimizer = getattr(torch.optim, args.optim)(params=[{'params': biases, 'lr': 2 * lr}, 
                                                            {'params': not_biases}],
                                                     lr=lr,
                                                     weight_decay=args.weight_decay)
    else:
        optimizer = getattr(torch.optim, args.optim)(params=model.parameters(),
                                                     lr=lr,
                                                     weight_decay=args.weight_decay)

else:
    checkpoint = torch.load(checkpoint)
    start_epoch = checkpoint['epoch'] + 1
    print('\nLoaded checkpoint from epoch %d.\n' % start_epoch)
    model = checkpoint['model']
    optimizer = checkpoint['optimizer']


model = model.to(device)
criterion = object_detection.SDSMultiBoxLoss(
    priors_cxcy=model.priors_cxcy, 
    usages_seg_feats=model.usages_seg_feats,
    get_separate_loss=args.get_separate_loss).to(device)

Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to /root/.cache/torch/hub/checkpoints/vgg16_bn-6c64b313.pth




Loaded base model.



/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


## dataset init

In [10]:
workers = 4
train_dataset = dataset.KaistPDDataset(ch_option=args.ch_option,
                                       is_sds=args.is_sds)
train_loader = torch.utils.data.DataLoader(train_dataset, 
                                           batch_size=args.train_batch_size, 
                                           shuffle=True,
                                           collate_fn=dataset.sds_collate_fn,
                                           num_workers=workers,
                                           pin_memory=True)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


## experiment

In [11]:
checkpoint_dir = '/content/drive/MyDrive/2021.summer_URP/PD/checkpoint'
checkpoint_path = os.path.join(checkpoint_dir,
                               str(args.baselineID)+'.pth.tar')

In [12]:
epochs = args.epochs
decay_lr_at = args.decay_lr_at
save_freq = 5
eval_freq = 10

# Epochs
for epoch in range(start_epoch, epochs+1):
    print(f"# ====== Epoch {epoch} ====== # {datetime.now()}")
    # Decay learning rate at particular epochs
    if epoch in decay_lr_at:
        adjust_learning_rate(optimizer, args.decay_lr_to)

    # One epoch's training
    train(train_loader=train_loader, model=model,
          criterion=criterion, optimizer=optimizer, run=run)
    if epoch % save_freq == 0:
        save_checkpoint(epoch, model, optimizer, checkpoint_path)

    if epoch % eval_freq == 0:
        eval.evaluate(model, ch_option=args.ch_option)

# ====== Epoch 1 ====== # 2021-08-06 02:16:30.231787


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[0/273]	Batch Time 113.973 (113.973)	Data Time 108.186 (108.186)	Loss 13.6309 (13.6309)	
[68/273]	Batch Time 30.969 (10.437)	Data Time 30.425 (9.819)	Loss 4.7058 (6.3981)	
[136/273]	Batch Time 31.195 (9.621)	Data Time 30.659 (9.041)	Loss 4.9432 (5.6098)	
[204/273]	Batch Time 19.442 (9.282)	Data Time 18.902 (8.714)	Loss 4.7205 (5.3262)	
[272/273]	Batch Time 5.220 (9.073)	Data Time 0.629 (8.497)	Loss 4.4965 (5.1662)	
# ====== Epoch 2 ====== # 2021-08-06 02:57:47.112449
[0/273]	Batch Time 4.184 (4.184)	Data Time 3.553 (3.553)	Loss 4.4392 (4.4392)	
[68/273]	Batch Time 2.039 (1.050)	Data Time 1.419 (0.425)	Loss 4.5485 (4.5621)	
[136/273]	Batch Time 0.968 (1.020)	Data Time 0.359 (0.398)	Loss 4.3482 (4.5074)	
[204/273]	Batch Time 0.613 (1.014)	Data Time 0.000 (0.391)	Loss 4.2518 (4.4534)	
[272/273]	Batch Time 0.468 (0.998)	Data Time 0.000 (0.377)	Loss 4.2481 (4.4000)	
# ====== Epoch 3 ====== # 2021-08-06 03:02:19.588313
[0/273]	Batch Time 3.937 (3.937)	Data Time 3.279 (3.279)	Loss 4.2619 (4.2

/content/URP_PD/object_detection.py:395: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:30.)
  image_boxes.append(class_decoded_locs[1 - suppress])
/content/URP_PD/object_detection.py:397: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:30.)
  image_scores.append(class_scores[1 - suppress])



Loading and preparing results...
DONE (t=0.02s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.46s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Miss Rate  (MR) @ Reasonable         [ IoU=0.50      | height=[55:10000000000] | visibility=[none+partial_occ] ] = 43.45%
Recall: 0.7752808988764045
# ====== Epoch 11 ====== # 2021-08-06 04:01:00.201216


...


Loading and preparing results...
DONE (t=0.02s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.32s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Miss Rate  (MR) @ Reasonable         [ IoU=0.50      | height=[55:10000000000] | visibility=[none+partial_occ] ] = 27.15%
Recall: 0.8230714791224345
# ====== Epoch 141 ====== # 2021-08-06 14:16:55.978867


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


[0/273]	Batch Time 4.683 (4.683)	Data Time 4.029 (4.029)	Loss 2.1078 (2.1078)	
[68/273]	Batch Time 2.016 (1.096)	Data Time 1.387 (0.473)	Loss 2.2297 (2.1023)	
[136/273]	Batch Time 1.905 (1.041)	Data Time 1.288 (0.415)	Loss 2.0415 (2.1270)	
[204/273]	Batch Time 1.952 (1.028)	Data Time 1.306 (0.403)	Loss 2.0283 (2.1245)	
[272/273]	Batch Time 0.470 (1.017)	Data Time 0.000 (0.393)	Loss 1.9479 (2.1211)	
# ====== Epoch 142 ====== # 2021-08-06 14:21:33.704207
[0/273]	Batch Time 4.862 (4.862)	Data Time 4.191 (4.191)	Loss 2.5390 (2.5390)	
[68/273]	Batch Time 1.982 (1.070)	Data Time 1.331 (0.443)	Loss 2.4495 (2.1071)	
[136/273]	Batch Time 0.630 (1.052)	Data Time 0.008 (0.426)	Loss 2.0457 (2.1237)	
[204/273]	Batch Time 0.628 (1.045)	Data Time 0.012 (0.420)	Loss 2.1639 (2.1247)	
[272/273]	Batch Time 0.470 (1.032)	Data Time 0.000 (0.408)	Loss 2.2879 (2.1245)	
# ====== Epoch 143 ====== # 2021-08-06 14:26:15.623427
[0/273]	Batch Time 5.107 (5.107)	Data Time 4.459 (4.459)	Loss 2.1643 (2.1643)	
[68/273

/content/URP_PD/object_detection.py:395: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:30.)
  image_boxes.append(class_decoded_locs[1 - suppress])
/content/URP_PD/object_detection.py:397: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:30.)
  image_scores.append(class_scores[1 - suppress])



Loading and preparing results...
DONE (t=0.02s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.31s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Miss Rate  (MR) @ Reasonable         [ IoU=0.50      | height=[55:10000000000] | visibility=[none+partial_occ] ] = 27.72%
Recall: 0.8192771084337349
